### INSTRUCTIONS AND RECOMMENDATIONS FOR CREATING YOUR OWN DATABASE FOR PHYLOGENETIC STUDIES

A database for phylogenetic study must contain:
- a BLAST database with the chosen proteomes;
- a file linking a gene to its protein products (g2r.tsv);
- a file linking a taxon identifier to scientific name of an organism (names.dmp).

We are going to get the most complete proteomes from the RefSeq database, so if you know you will be using your own set of proteomes, please, modify the algorithm accordingly. Moreover, we will need BLAST+ as well as Entrez Direct utilities.

Note that the algorithm is not intended for studying prokaryotic genes since these proteomes often use non-redundant protein accession numbers (with the prefix 'WP_').

#### 1. Getting the necessary files.

First, download the current proteomes list from RefSeq with the corresponding BUSCO completeness scores (you will need Entrez Direct installed).

In [ ]:
%%bash

mkdir refseq_proteomes
cd refseq_proteomes
esearch -db assembly -query 'has_egap_annotation[prop] AND "latest refseq"[filter]' \
  | esummary \
  | xtract \
    -pattern DocumentSummary \
    -element \
      AssemblyAccession,\
      Organism,\
      Taxid,\
      Busco/BuscoLineage,\
      Busco/TotalCount,\
      Busco/Complete,\
      Busco/SingleCopy,\
      Busco/Duplicated,\
      Busco/Fragmented,\
      Busco/Missing \
> busco_scores.tsv

Then, get the taxdump.tar.gz.

In [ ]:
%%bash

cd refseq_proteomes
mkdir taxonomy
cd taxonomy
wget -q ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz -O taxdump.tar.gz 
sleep 1
tar -xzf taxdump.tar.gz
rm taxdump.tar.gz
rm citations.dmp
rm delnodes.dmp
rm division.dmp
rm gencode.dmp
rm images.dmp
rm merged.dmp
rm readme.txt
rm gc.prt

#### 2. Find the most complete proteomes.

We don't need to take all proteomes into the analysis: it is sufficient to take in several most complete proteomes from each class or another taxonomic rank.

We will take 3 most complete proteomes from classes.

Lets load nodes.dmp and names.dmp for managing taxonomic ranks:

In [ ]:
import pandas as pd

sep = '\t\|\t'

nodes = pd.read_table(
    'refseq_proteomes/taxonomy/nodes.dmp',
    sep = sep,
    header = None,
    engine ='python'
)

nodes.columns = [
	'Taxid',
 	'Parent',
 	'Rank',
 	'EMBL',
 	'Division',
 	'Inherited_div',
 	'Gencode',
 	'Inherited_gencode',
 	'Mito',
 	'Inherited_mito',
 	'GenBank_hidden',
 	'Hidden_subtree',
 	'Comments'
]

names = pd.read_table(
    'refseq_proteomes/taxonomy/names.dmp',
    sep = sep,
    header = None,
    engine ='python'
)

names.columns = [
	'Taxid',
	'Name',
	'Unique',
	'Class'
]


Saving all identifiers of classes:

In [ ]:
classes_ids = nodes[nodes['Rank'] == 'class']['Taxid'].to_list()

We also have to use a sorted (by a complete BUSCO score -- column 5 -- in our case) file with BUSCO scores:

In [ ]:
busco = pd.read_table(
    'refseq_proteomes/busco_scores.tsv',
    header = None,
    sep = '\t'
)

busco.columns = [
    'GCF',
    'Name',
    'Taxid',
    'Lineage',
    'Count',
    'Score',
    'Single',
    'Dupl',
    'Fragm',
    'Miss'
]
busco = busco.dropna()
busco = busco.sort_values(by = ['Score'], ascending = False)

Lets use the taxid column as a list:

In [ ]:
def find_class(current_taxid):
    if not (current_taxid in classes_ids):
        current_taxid = nodes[nodes['Taxid'] == current_taxid]['Parent'].to_list()
        if len(set(current_taxid)) > 1:
            raise Exception('Taxid', current_taxid, 'has multiple parents')
        elif len(set(current_taxid)) == 0:
            raise Exception('Warning! Root-tracing failed')
        elif current_taxid[0] == 1:
            return 0
        else:
            return find_class(current_taxid[0])
    else:
        return current_taxid

busco['Class'] = busco['Taxid'].map(find_class).astype(int)
print(len(busco['Class'].unique()))

There were 36 classes in total, and we are going to obtain up to 3*36=111 proteomes. Note that 0s indicate organisms that do not have class according to the nodes.dmp file. Lets see what these are.

In [ ]:
busco[busco['Class'] == 0]

In the original paper, we have used other taxonomic ranks for these species. Now we are going to just take the top 3 proteomes with 0s, as well as Latimeria chalumnae and Protopterus annectens and human.

In [ ]:
proteomes_list = [
    busco[busco['Taxid'] == 7897]['GCF'].to_list()[0],
    busco[busco['Taxid'] == 7888]['GCF'].to_list()[0],
    busco[busco['Taxid'] == 9606]['GCF'].to_list()[0]
]
# The top 3 with 0s will be added later

Now lets just take the top 3 proteomes in each class:

In [ ]:
for name, class_df in busco.groupby('Class'):
    class_df = class_df.sort_values(by = ['Score'], ascending = False)
    class_proteomes = list()
    taxids = set()
    for index, row in class_df.iterrows():
        if not (row['Taxid'] in taxids):
            class_proteomes.append(row['GCF'])
        taxids.add(row['Taxid'])
    proteomes_list += class_proteomes[:3]

Lets write down obtained assembly identifiers.

In [ ]:
with open('./refseq_proteomes/assembly_ids.txt', 'w') as out:
    out.write('\n'.join(list(set(proteomes_list))))

#### 3. Download proteomes, create and configure your BLAST database.

We will be using the NCBI Datasets tool.

In [ ]:
%%bash

cd refseq_proteomes
mkdir assembly_files
cd assembly_files

datasets download genome accession \
    --inputfile ../assembly_ids.txt \
    --include protein,gff3 \
    --no-progressbar \
    --dehydrated

unzip ncbi_dataset.zip
rm ncbi_dataset.zip
rm README.md

The next step is going to be **long**.

In [ ]:
%%bash

cd refseq_proteomes/assembly_files

datasets rehydrate --no-progressbar --directory .

Lets manage the obtained files and concatenate all FASTA files and feature tables.

In [ ]:
%%bash

cd ./refseq_proteomes/assembly_files
mkdir ../busco_refseq

# Making a single FASTA file
> ../busco_refseq/busco_refseq.fasta
for fasta_file in ./ncbi_dataset/data/*/protein.faa
do
  cat $fasta_file >> ../busco_refseq/busco_refseq.fasta
done

In [ ]:
import os
from glob import glob

gff_path = 'refseq_proteomes/assembly_files/ncbi_dataset/data/*/genomic.gff'
feature_table_path = 'refseq_proteomes/busco_refseq/busco_feature_table.txt'

with open(feature_table_path, 'w') as ftable:
    ftable.write('\t'.join([
        'assembly',
        'product_accession',
        'symbol',
        'GeneID'
    ]) + '\n')
    for filename in glob(gff_path):
        assembly = os.path.split(os.path.split(filename)[0])[1]
        product_set = set()
        with open(filename, 'r') as gff_file:
            gff = gff_file.readlines()
            gff = [i for i in gff if 'protein_id=' in i]
            for i in gff:
                i = i.replace(',', ';')
                product = i.split('protein_id=')[1].split(';')[0].strip()
                symbol = i.split('gene=')[1].split(';')[0].strip()
                gID = i.split('GeneID:')[1].split(';')[0].strip()
                product_set.add('\t'.join([
                    assembly,
                    product,
                    symbol,
                    gID
                ]) + '\n')
            for i in product_set:
                ftable.write(i)

Checking if FASTA files and the feature table are fully compatible...

In [ ]:
import gzip

ft = pd.read_csv('refseq_proteomes/busco_refseq/busco_feature_table.txt', sep = '\t')

assemblies = set()
for path in glob('refseq_proteomes/assembly_files/ncbi_dataset/data/*/*'):
    assembly = os.path.split(os.path.split(path)[0])[1]
    assemblies.add(assembly)

if assemblies == set(ft['assembly']) == set(proteomes_list):
    print("Assemblies list: ok")
else:
    raise Exception("FASTA file and assembly list do not match")

for path in glob('refseq_proteomes/assembly_files/*/*.faa.gz'):
    filename = os.path.basename(path)
    with gzip.open(path, 'r') as inp:
        proteome = inp.readlines()
    proteome = [p.decode('utf-8').split()[0][1:] for p in proteome if p.startswith(b'>')]
    assembly = '_'.join(filename.split('_')[0:2])
    curr_df = ft[ft['assembly'] == assembly]
    if set(proteome) == set(curr_df['product_accession']):
        print(assembly + ': ok')
    else:
        raise Exception(assembly + ': protein accession numbers from the feature table and FASTA files do not match')

Adding the taxonomy information to the feature table.

In [ ]:
busco = busco[['GCF', 'Taxid']]

taxid_dict = dict()
for index, b in busco.iterrows():
    taxid_dict[b['GCF']] = b['Taxid']

ft['#tax_id'] = ft['assembly'].map(taxid_dict)

if len(ft['#tax_id'].unique()) != len(ft['assembly'].unique()):
    raise Exception('Check the tax_id and assembly correspondence')

g2r = ft

g2r = g2r.rename(columns = {
    'product_accession': 'protein_accession.version',
    'symbol': 'Symbol'
})

g2r.to_csv('refseq_proteomes/g2r.tsv', sep = '\t', index = False)

Clearing up space...

In [ ]:
%%bash

cd refseq_proteomes
rm -r assembly_files

Finally, lets create a BLAST search database (provide path to your makeblastdb binary if necessary).

In [ ]:
%%bash

cd ./refseq_proteomes/busco_refseq
makeblastdb \
  -dbtype prot \
  -in ./busco_refseq.fasta \
  -title busco_refseq \
  -parse_seqids \
  -out busco_refseq \
  -max_file_sz 2GB

Edit the .ncbirc file accordingly (insert your paths if necessary).

In [ ]:
%%bash

busco_refseq_path=$(readlink -f ./refseq_proteomes/busco_refseq)
echo "[BLAST]" > ~/.ncbirc
echo "BLASTDB=${busco_refseq_path}" >> ~/.ncbirc
echo "DATA_LOADERS=blastdb" >> ~/.ncbirc
echo "BLASTDB_PROT_DATA_LOADER=busco_refseq" >> ~/.ncbirc

Edit the configuration file (replace blastdbcmd_path and blastp_path if necessary).

In [ ]:
%%bash

refseq_proteomes_path=$(readlink -f ./refseq_proteomes)
blastdbcmd_path=$(which blastdbcmd)
blastp_path=$(which blastp)
echo "# Local 'gene2refseq' file" > ./cogconf.txt
echo "path2G2R:${refseq_proteomes_path}/g2r.tsv" >> ./cogconf.txt
echo "# If you use refseq, download this file and unzip, then provide names.dmp file: ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz" >> ./cogconf.txt
echo "path2T2N:${refseq_proteomes_path}/taxonomy/names.dmp" >> ./cogconf.txt
echo "# Name of database with representative taxids" >> ./cogconf.txt
echo "databaseName:busco_refseq" >> ./cogconf.txt
echo "# Path to Blastp utility" >> ./cogconf.txt
echo "path2blastp:${blastp_path}" >> ./cogconf.txt
echo "# Path to BlastDBCmd utility" >> ./cogconf.txt
echo "blastdbcmd:${blastdbcmd_path}" >> ./cogconf.txt

Your database is ready for work! Please, create a conda environment for this tool:
```
conda env create -n [NEW_ENV_NAME] -f dependencies.yml
```
...activate it...
```
conda activate [NEW_ENV_NAME]
```
 and test your installation (the algorithm uses 40 threads by default; enter your value in the "-t" parameter -- we use 10 threads in this example):

In [70]:
%%bash

echo "NP_001104262" > ./refseq_proteomes/test_input.txt
# Optional: uncomment the next line to analyze two genes one by one.
# echo "NP_001278" >> ./refseq_proteomes/test_input.txt

# Observe the test_input.txt for input file example

python3 ./cog.py \
    ./refseq_proteomes/test_input.txt \
    ./refseq_proteomes/test_output \
    -t 10